In [41]:
import pandas as pd
import matplotlib.pyplot as plt
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle

In [42]:
foundation_folder = "FoodData_Central_foundation_food_csv_2023-10-26"

foundation_food_df = pd.read_csv(f'data_source/{foundation_folder}/food.csv')
foundation_food_nutrient_df = pd.read_csv(f'data_source/{foundation_folder}/food_nutrient.csv')
foundation_nutrient_df = pd.read_csv(f'data_source/{foundation_folder}/nutrient.csv')

C:\Users\Michael\AppData\Local\Temp\ipykernel_44080\2253104221.py:4: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  foundation_food_nutrient_df = pd.read_csv(f'data_source/{foundation_folder}/food_nutrient.csv')


In [43]:
foundation_food_df.info()
# remove description is null data
foundation_food_df = foundation_food_df[foundation_food_df.description.notna()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60432 entries, 0 to 60431
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fdc_id            60432 non-null  int64  
 1   data_type         60432 non-null  object 
 2   description       60424 non-null  object 
 3   food_category_id  58732 non-null  float64
 4   publication_date  60432 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 2.3+ MB


In [44]:
foundation_food_nutrient_df.info()
# remove amount is null data
foundation_food_nutrient_df = foundation_food_nutrient_df[foundation_food_nutrient_df.amount.notna()]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140824 entries, 0 to 140823
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 140824 non-null  int64  
 1   fdc_id             140824 non-null  int64  
 2   nutrient_id        140824 non-null  int64  
 3   amount             140804 non-null  float64
 4   data_points        137625 non-null  float64
 5   derivation_id      140681 non-null  float64
 6   min                13825 non-null   float64
 7   max                13825 non-null   float64
 8   median             14427 non-null   float64
 9   footnote           3 non-null       object 
 10  min_year_acquired  28689 non-null   object 
dtypes: float64(6), int64(3), object(2)
memory usage: 11.8+ MB


In [45]:
foundation_nutrient_df.info()
# only keep the target nutrition I want to track
tracking_nutrition = {"Protein", "Total lipid (fat)", "Carbohydrate, by difference"}
foundation_nutrient_df = foundation_nutrient_df[foundation_nutrient_df.name.isin(tracking_nutrition)].rename(columns={'id': 'nutrient_id'})

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            477 non-null    int64  
 1   name          477 non-null    object 
 2   unit_name     477 non-null    object 
 3   nutrient_nbr  465 non-null    float64
 4   rank          466 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 18.8+ KB


In [46]:
foundation_result_df = foundation_food_nutrient_df.merge(foundation_nutrient_df, on="nutrient_id", how="left")
foundation_result_df = foundation_result_df[foundation_result_df.name.notna()]
foundation_result_df = foundation_result_df.groupby(['fdc_id', 'name'])['amount'].sum().unstack().reset_index("fdc_id")

In [47]:
foundation_result_df = foundation_result_df.merge(foundation_food_df, on="fdc_id", how="inner")
foundation_result_df = foundation_result_df[["fdc_id", "description", "Carbohydrate, by difference", "Protein", "Total lipid (fat)"]]

In [48]:
foundation_result_df = foundation_result_df.dropna(how='any')

In [49]:
foundation_result_df = foundation_result_df.groupby("description")[['Carbohydrate, by difference', "Protein", "Total lipid (fat)"]].mean().reset_index()

In [50]:
# fill 0 if any nutrition value < 0 (base on observation from https://fdc.nal.usda.gov/fdc-app.html#/)
for i in tracking_nutrition:
    foundation_result_df[i] = foundation_result_df[i].mask(foundation_result_df[i] < 0, 0)
    foundation_result_df[i] = foundation_result_df[i].round(2)

In [51]:
# 創建 PDF 文件
pdf_filename = "gpt_assistant_input/foundation_food_nutrition.pdf"
pdf = SimpleDocTemplate(pdf_filename, pagesize=letter)
data = [foundation_result_df.columns[:,].values.astype(str)] + foundation_result_df.values.tolist()

# 創建表格
t = Table(data)

# 添加表格樣式
t.setStyle(TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.grey),
                       ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                       ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                       ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                       ('BOTTOMPADDING', (0, 0), (-1, 0), 2),
                       ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                       ('GRID', (0, 0), (-1, -1), 1, colors.black),
                       ('FONTSIZE', (0, 0), (-1, -1), 6)]))

# 將表格添加到 PDF 文件
pdf.build([t])

print(f"PDF file saved as {pdf_filename}")


PDF file saved as gpt_assistant_input/foundation_food_nutrition.pdf
